In [1]:
import re 
import pandas as pd
import numpy as np
import emoji
from collections import Counter
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk

In [2]:
def date_time(s):
    pattern = '^([0-9]+)(\/)([0-9]+)(\/)([0-9]+), ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -'
    result = re.match(pattern, s)
    return bool(result)

# Check if a string is a contact message
def find_contact(s):
    s = s.split(":")
    return len(s) == 2

# Extract date, time, author, and message from a line
def getMessage(line):
    splitline = line.split(' - ')
    datetime = splitline[0]
    datetime_parts = datetime.split(', ')
    if len(datetime_parts) == 2:
        date, time = datetime_parts
    else:
        date, time = None, None
    message = " ".join(splitline[1:])
    if find_contact(message):
        splitmessage = message.split(": ")
        author = splitmessage[0]
        message = " ".join(splitmessage[1:])
    else:
        author = None
    return date, time, author, message



In [6]:
data = []
conversation = 'abc.txt'
with open(conversation, encoding="utf-8") as fp:
    fp.readline()
    messageBuffer = []
    date, time, author = None, None, None
    while True:
        line = fp.readline()
        if not line:
            break
        line = line.strip()
        if date_time(line):
            if messageBuffer:
                data.append([date, time, author, ' '.join(messageBuffer)])
            messageBuffer.clear()
            date, time, author, message = getMessage(line)
            messageBuffer.append(message)
        else:
            messageBuffer.append(line)
        

In [9]:
df = pd.DataFrame(data, columns=["Date", "Time", "Contact", "Message"])
df['Date'] = pd.to_datetime(df['Date'])
data = df.dropna()

nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiments = SentimentIntensityAnalyzer()
data["positive"] = [sentiments.polarity_scores(i)["pos"] for i in data["Message"]]
data["negative"] = [sentiments.polarity_scores(i)["neg"] for i in data["Message"]]
data["neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data["Message"]]

data.head(0)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\12nik\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,Date,Time,Contact,Message,positive,negative,neutral


In [5]:
x=sum(data["positive"])
y=sum(data["negative"])
z=sum(data["neutral"])

def score(a,b,c):
    if (a>b) and (a>c):
        print("positive ")
    if (b>a) and (b>c):
        print("negative ")
    if (c>a) and (c>b):
        print("neutral ")

score(x,y,z)

neutral 
